In [2]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions
from debiaswe.data import load_gender_seed

E = WordEmbedding('./embeddings/fasttext_wiki-news-300d-1M.vec') # This may take a while, 1M words

In [ ]:
# load professions
professions = load_professions()
profession_words = [p[0] for p in professions]

# gender direction
v_gender = E.diff('she', 'he')

# Uncomment below for direction based on multiple definitional pairs
# with open('./data/definitional_pairs.json', "r") as f:
#     defs = json.load(f)
# v_gender = we.doPCA(defs, E).components_[0]

# analogies gender
a_gender = E.best_analogies_dist_thresh(v_gender, thresh=0.8, max_words=5000)
we.viz(a_gender)

In [ ]:
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

In [ ]:
from debiaswe.debias import hard_debias

# Lets load some gender related word lists to help us with debiasing
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
print("definitional", defs)

with open('./data/equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f)

with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

hard_debias(E, gender_specific_words, defs, equalize_pairs)

In [ ]:
# analogies gender
a_gender_debiased = E.best_analogies_dist_thresh(v_gender, thresh=0.8, max_words=5000)
we.viz(a_gender_debiased)

In [ ]:
# profession analysis gender
sp_debiased = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp_debiased[0:20], sp_debiased[-20:]